In [2]:
#=
Run this cell so you can use the necessary package(s)
and provided functions
=#

using LinearAlgebra, Random, Plots

function backwardsub(U, b)
    n = length(b)
    x = Vector{Float64}(undef, n) 
    x[n] = b[n]/U[n,n]
    for i in n-1:-1:1
        x[i]=(b[i]- (U[i,(i+1):n])' *x[(i+1):n] )/U[i,i]
    end
    return x
end

function forwardsub(L, b)
    n = length(b)
    x = Vector{Float64}(undef, n); 
    x[1] = b[1]/L[1,1] 
    for i = 2:n 
        x[i]=(b[i]- (L[i,1:i-1])' *x[1:i-1] )/L[i,i] 
    end
    return x
end

forwardsub (generic function with 1 method)

In [3]:
#Run me & read me!

function gradHess(f,x0) 
    n=size(x0,1)
    H=zeros(n,n) #matrix to hold the Hessian
    grad1=zeros(1,n) #vector to hold the gradient
    
    Id=diagm(0=>fill(1., size(H,1))) # Another way to make an identity matrix
    
    delta=0.01
    h=delta
    
    for i=1:n
        grad1[i]=(f(x0+ h*Id[:,i]) -f(x0 -  h*Id[:,i]))[1]/(2*h)
        for j=1:n
            H[i,j]= (f(x0+ h*Id[:,i] + delta*Id[:,j]) 
                    - f(x0+  h*Id[:,i]-delta*Id[:,j]) 
                    - f(x0-  h*Id[:,i] + delta*Id[:,j])
                    + f(x0- h*Id[:,i]-delta*Id[:,j]))[1]/(4*h*delta)
        end
    end
    return  grad1, H
end

gradHess (generic function with 1 method)

In [4]:
#=
I am creating a super complicated function to show you
how efficiently gradHess() can compute the gradient 
and the Hessian.
=#

Random.seed!(4321);
n=20;
A2=rand(n,n);
A4=rand(n,n);
f2(x)= x'*A2*x + x'*x*x'*A4*x;
x0=rand(n,1)

# READ ME!!!
# Here is an example using the function gradHess  

(gradN, Hess) =gradHess(f2, x0)

println("gradN is the gradient vector")

#This is a row vector!
show(stdout, "text/plain", gradN) 

println("\n \n Hess is the Hessian")
show(stdout, "text/plain", Hess)

gradN is the gradient vector
1×20 Matrix{Float64}:
 64.4735  66.1955  59.5686  62.431  63.9542  68.5768  46.7627  68.8214  79.43  59.0365  75.9139  74.7611  61.2015  67.3749  97.0458  89.0346  104.819  75.5183  59.385  49.4603
 
 Hess is the Hessian
20×20 Matrix{Float64}:
 82.4927   17.8681  15.1615   14.9482   13.3876  19.2247   8.97361  19.4722  21.5064  11.6236  18.2139   19.1615  15.6989  19.2355  24.0181  24.0053   26.0827  17.5149  10.8437   11.7912
 17.8681   77.1881  15.3837   17.6602   16.5884  18.178   13.2542   20.5596  22.2138  13.2045  20.6064   19.7203  13.7665  19.4991  21.1748  18.5585   23.2364  18.1261  19.3072   16.603
 15.1615   15.3837  80.8402   12.2898   12.937   19.2018   8.88778  12.0637  19.3283  16.137   19.3201   19.4254  13.5332  14.8012  20.7244  22.0363   24.7264  16.9266  12.2871   12.8142
 14.9482   17.6602  12.2898   75.2402   13.5157  17.7585   6.27503  16.7606  20.1902  12.5715  18.8279   16.867   11.7757  18.0904  31.0703  23.1486   28.2957  17.0342

### Your Task



Use `gradHess(f,x0)` to minimize a function `g(x)` where x \in \mathbb{R}^{20}. Your minimizer should be named `x_star` and your minimum value should be named `gMin`. **Do not write a function, just build a while loop.**



You are allowed to use the `</code> operator.`



In [5]:
#= 
this cell declares g(x), a function that 
depends on 20 variables and xk for k=0, 
which you are to use as your starting point
=#
n=20 

Random.seed!(4321); 
A2=rand(n,n) 
A4=rand(n,n) 

g(x)= -x'*A2'*A2*x + x'*x*x'*A4'*A4*x
k=0
xk=100*rand(n,1)-200*rand(n,1)

tol = 1e-6

1.0e-6

In [8]:
# YOUR ANSWER HERE
(gradN, Hess) =gradHess(g, xk)
while abs(norm(gradN)) > tol
    F = lu(Hess)
    y = forwardsub(F.L, F.P * (-gradN)')
    delta_x = backwardsub(F.U, y)
    xk = xk + delta_x
    (gradN, Hess) =gradHess(g, xk)
end

x_star = xk
gMin = g(x_star)
#Checking your minimum
@assert(isapprox(gMin[1], -25.779955962689826, atol = 1E-4))
#checking your x vector
x_ans = [-0.12476569386937346
 -0.18835490238291708
 -0.10405520076793454
 -0.1474839335609496
 -0.0850119225231768
 -0.062197653846722586
 -0.1603093008601981
 -0.13992352400773628
 -0.1343513953084501
 -0.22533654803008207
 -0.17944614988326335
 -0.19649613612403263
 -0.21012326594871955
 -0.11542489619574255
 -0.32149679187873076
 -0.09496475719863778
 -0.17356879734348407
 -0.11054979069631582
 -0.12179310436225206
 -0.2042537017066612]
@assert(isapprox(x_star, x_ans, atol = 1E-4))